# [Advent of Code 2021: Day 4 - Giant Squid](https://adventofcode.com/2021/day/4)
[puzzle input](https://adventofcode.com/2021/day/4/input)

### Part One

You're already almost 1.5km (almost a mile) below the surface of the ocean, already so deep that you can't see any sunlight. What you **can** see, however, is a giant squid that has attached itself to the outside of your submarine.

Maybe it wants to play [bingo](https://en.wikipedia.org/wiki/Bingo_(American_version))?

Bingo is played on a set of boards each consisting of a 5x5 grid of numbers. Numbers are chosen at random, and the chosen number is **marked** on all boards on which it appears. (Numbers may not appear on all boards.) If all numbers in any row or any column of a board are marked, that board **wins**. (Diagonals don't count.)

The submarine has a **bingo subsystem** to help passengers (currently, you and the giant squid) pass the time. It automatically generates a random order in which to draw numbers and a random set of boards (your puzzle input). For example:

```
7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1

22 13 17 11  0
 8  2 23  4 24
21  9 14 16  7
 6 10  3 18  5
 1 12 20 15 19

 3 15  0  2 22
 9 18 13 17  5
19  8  7 25 23
20 11 10 24  4
14 21 16 12  6

14 21 17 24  4
10 16 15  9 19
18  8 23 26 20
22 11 13  6  5
 2  0 12  3  7

```

After the first five numbers are drawn (`7`, `4`, `9`, `5`, and `11`), there are no winners, but the boards are marked as follows (shown here adjacent to each other to save space):

**`22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7`**

After the next six numbers are drawn (`17`, `23`, `2`, `0`, `14`, and `21`), there are still no winners:

**`22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7`**

Finally, `24` is drawn:

**`22 13 17 11  0         3 15  0  2 22        14 21 17 24  4
 8  2 23  4 24         9 18 13 17  5        10 16 15  9 19
21  9 14 16  7        19  8  7 25 23        18  8 23 26 20
 6 10  3 18  5        20 11 10 24  4        22 11 13  6  5
 1 12 20 15 19        14 21 16 12  6         2  0 12  3  7`**

At this point, the third board **wins** because it has at least one complete row or column of marked numbers (in this case, the entire top row is marked: **`14 21 17 24 4`**).

The **score** of the winning board can now be calculated. Start by finding the **sum of all unmarked numbers** on that board; in this case, the sum is `188`. Then, multiply that sum by **the number that was just called** when the board won, `24`, to get the final score, **`188 * 24 = 4512`**.

To guarantee victory against the giant squid, figure out which board will win first. **What will your final score be if you choose that board?**

In [1]:
import unittest
import re
import numpy as np
from IPython.display import Markdown, display

from aoc_puzzle import AocPuzzle

class Bingo(AocPuzzle):
    
    def load_draws(self, data):
        self.draws = list(map(int, data.split(',')))
        
    def load_boards(self, data):
        self.boards = []
        for board in data:
            board_lines = board.split('\n')
            board_data = []
            for line in board_lines:
                row = list(map(int, re.split('\s+', line.strip())))
                row = [(item, False) for item in row] 
                board_data.append(row)
            self.boards.append(board_data)
    
    def parse_data(self, raw_data):
        data_sets = raw_data.split('\n\n')
        self.load_draws(data_sets[0])
        self.load_boards(data_sets[1:])
        
    def mark_boards(self, val):
        for b, board in enumerate(self.boards):
            for r, row in enumerate(board):
                for i, (num, mark) in enumerate(row):
                    if num == val:
                        self.boards[b][r][i] = (num, True)
                        
    def is_winner(self, board):
        vals, marks = np.transpose(board)
        for col in marks:
            if col.all():
                return True
        for row in np.transpose(marks):
            if row.all():
                return True
        return False
    
    def get_winners(self):
        winners = []
        for b, board in enumerate(self.boards):
            if self.is_winner(board):
                winners.append(b)
        return winners
                
    def get_board_checksum(self, board_num):
        checksum = 0
        for row in self.boards[board_num]:
            for val, mark in row:
                if mark == False:
                    checksum += val
        return checksum                
        
    def play(self):
        winner = -1
        for draw in self.draws:
            self.mark_boards(draw)
            winner = self.get_winners()
            if winner:
                break
        return (draw, winner[0])
            
            
    def run(self, output=False):
        last_draw, winning_board = self.play()
        
        if winning_board == -1:
            result = -1
            if output:
                display(Markdown(f'### No Winner Found: `{result}`'))
        else:
            check_sum = self.get_board_checksum(winning_board)
            result = last_draw * check_sum
        if output:
            display(Markdown(f'### Result is `{result}`'))
            
        return result
        

class TestBasic(unittest.TestCase):

    def test_parse_data(self):
        in_data = '7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1\n\n22 13 17 11  0\n 8  2 23  4 24\n21  9 14 16  7\n 6 10  3 18  5\n 1 12 20 15 19'
        exp_out_draws = [7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1]
        exp_out_boards = [[[(22,False),(13,False),(17,False),(11,False),(0,False)],[(8,False),(2,False),(23,False),(4,False),(24,False)],[(21,False),(9,False),(14,False),(16,False),(7,False)],[(6,False),(10,False),(3,False),(18,False),(5,False)],[(1,False),(12,False),(20,False),(15,False),(19,False)]]]
        bingo = Bingo(in_data)
        self.assertEqual(bingo.draws, exp_out_draws)
        self.assertEqual(bingo.boards, exp_out_boards)
        
    def test_puzzle(self):
        input_data = ['7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1\n\n22 13 17 11  0\n 8  2 23  4 24\n21  9 14 16  7\n 6 10  3 18  5\n 1 12 20 15 19\n\n 3 15  0  2 22\n 9 18 13 17  5\n19  8  7 25 23\n20 11 10 24  4\n14 21 16 12  6\n\n14 21 17 24  4\n10 16 15  9 19\n18  8 23 26 20\n22 11 13  6  5\n 2  0 12  3  7']
        exp_output = [4512]
        for in_data, exp_out in tuple(zip(input_data, exp_output)):
            bingo = Bingo(in_data)
            self.assertEqual(bingo.run(), exp_out)
        
unittest.main(argv=[""], exit=False)

..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


In [2]:
bingo = Bingo("input/d04.txt")
bingo.run(output=True)

### Result is `27027`

27027

### Part Two

On the other hand, it might be wise to try a different strategy: let the giant squid win.

You aren't sure how many bingo boards a giant squid could play at once, so rather than waste time counting its arms, the safe thing to do is to **figure out which board will win last** and choose that one. That way, no matter which boards it picks, it will win for sure.

In the above example, the second board is the last to win, which happens after `13` is eventually called and its middle column is completely marked. If you were to keep playing until this point, the second board would have a sum of unmarked numbers equal to `148` for a final score of **`148 * 13 = 1924`**.

Figure out which board will win last. **Once it wins, what would its final score be?**

In [3]:
class Bingo2(Bingo):
    def remove_boards(self, remove_list):
        trimmed = []
        for b, board in enumerate(self.boards):
            if b not in remove_list:
                trimmed.append(board)
        self.boards = trimmed
    
    def play(self):
        for draw in self.draws:
            self.mark_boards(draw)
            winners = self.get_winners()
            if winners:
                if len(self.boards) == 1:
                    break
                else:
                    self.remove_boards(winners)
        return (draw, 0)
    
class TestBasic(unittest.TestCase):
        
    def test_puzzle2(self):
        input_data = ['7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1\n\n22 13 17 11  0\n 8  2 23  4 24\n21  9 14 16  7\n 6 10  3 18  5\n 1 12 20 15 19\n\n 3 15  0  2 22\n 9 18 13 17  5\n19  8  7 25 23\n20 11 10 24  4\n14 21 16 12  6\n\n14 21 17 24  4\n10 16 15  9 19\n18  8 23 26 20\n22 11 13  6  5\n 2  0 12  3  7']
        exp_output = [1924]
        for in_data, exp_out in tuple(zip(input_data, exp_output)):
            bingo = Bingo2(in_data)
            self.assertEqual(bingo.run(), exp_out)
        
unittest.main(argv=[""], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.006s

OK


In [4]:
bingo = Bingo2("input/d04.txt")
bingo.run(output=True)

### Result is `36975`

36975